#사용하는 함수 import, 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, confusion_matrix, recall_score
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler
import xgboost
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.linear_model import SGDClassifier
import warnings

secom = pd.read_csv('uci-secom.csv')
secom.shape

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


(1567, 592)

#label 분리, 상관관계 계수 낮은 열 제거, NaN값 제거

In [2]:
secom=secom.drop(columns='Time')
co = secom.corr()
co_x = co['Pass/Fail']

t = 0
a = np.zeros((1,1))
for i in co_x:
  if np.isnan(i):
    a = np.append(a,t)
  t = t+1
a = np.delete(a,0)

for i in a:
  i = int(i)
  secom = secom.drop([str(i)],axis=1)

secom.fillna(secom.mean(), inplace=True)

x = secom.iloc[:, 0:-1]
y = secom.iloc[:, -1]
x.shape

(1567, 474)

#StandardScaler 적용

In [3]:
ss = StandardScaler()
x = ss.fit_transform(x)

#GridSearch를 통해 Hyper parameter 탐색

In [ ]:
# 참고용, 실제 로직에서 사용하지 않음.
'''x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 13, stratify = y)
param_grid = {
    'eta0': [1e-1, 1e-2, 1e-3, 1e-5],
    'l1_ratio': [1e-1, 1e-2, 1e-3],
    'max_iter': [1e3, 1e5, 1e7],
}
estimator = SGDClassifier(loss = 'perceptron')

kf = KFold(random_state=13,
           n_splits=10,
           shuffle=True,
          )

grid_search = GridSearchCV(estimator=estimator, 
                           param_grid=param_grid, 
                           cv=kf, 
                           n_jobs=-1
                          )

grid_search.fit(x_train, y_train)
print(grid_search.best_params_)'''

#실제 학습 및 평가

In [5]:
warnings.filterwarnings(action='ignore')

xgb_clf = xgboost.XGBClassifier(objective = 'binary:logistic', booster = 'gblinear', gamma=0, min_child_weight=0.01)
sgd_clf = SGDClassifier(loss = 'perceptron', l1_ratio = 1e-3, fit_intercept=True, max_iter=1e7, eta0 = 1e-7, shuffle=False, epsilon='DEFAULT_EPSILON', n_jobs=-1, learning_rate='constant', warm_start=True, validation_fraction=0.1)

auc_score_sgd = 0
for i in range(100):
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i, stratify = y)

  rus = RandomUnderSampler(random_state = 303)
  x_train_under,y_train_under = rus.fit_sample(x_train, y_train)
 
  xgb_clf.fit(x_train_under, y_train_under)
  xgb_proba = xgb_clf.predict_proba(x_test)

  sgd_clf.fit(x_train_under, y_train_under)
  sgd_pred = sgd_clf.predict(x_test)

  for i in sgd_pred:
    if sgd_pred[i] == -1:
      if xgb_proba[i][1] > 0.8:
        sgd_pred[i] = 1
    if sgd_pred[i] == 1:
      if xgb_proba[i][1] < 0.2:
        sgd_pred[i] = -1

  auc_score_sgd+=(roc_auc_score(y_test, sgd_pred))

print('ROC_AUC_SCORE : ', (auc_score_sgd)/100)

ROC_AUC_SCORE :  0.8500113765642776
